In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import preprocessing

In [62]:
data= pd.read_csv("playerStats.csv",index_col="Unnamed: 0")

data.head()

,matchID,Court,min_fromNet,avg_fromNet,max_fromNet,std_fromNet,min_fromCentre,avg_fromCentre,max_fromCentret,std_fromCentret,...,behindLine,distPF,yPF,xPF,cumDist,cumY,cumX,Winner,Gender,Server
0,1,1.0,257.303962,842.355322,1381.306329,384.882164,47.510460,141.997342,301.441752,88.165214,...,0.285714,16.396104,14.747000,3.796541,3672.727268,3303.327902,850.425163,0,1,1
1,1,0.0,118.382947,838.815764,1482.824234,542.643022,1.213837,151.834926,338.524431,102.532740,...,0.433036,11.850770,9.707570,4.276306,2654.572510,2174.495734,957.892586,1,1,0
2,2,1.0,1202.830806,1259.300150,1329.939874,32.602326,107.532377,260.033812,398.776271,106.736006,...,1.000000,13.190400,10.385537,5.663344,791.423975,623.132191,339.800651,0,1,0
3,2,0.0,978.857600,1157.642380,1241.160040,41.132159,22.166075,49.851511,69.591791,10.722623,...,0.200000,13.689481,12.823885,1.928863,821.368859,769.433111,115.731760,1,1,1
4,3,1.0,1200.101358,1334.875881,1456.258443,64.326507,0.298514,171.547100,426.579458,114.439209,...,1.000000,13.161627,8.896785,6.852655,3987.973053,2695.725933,2076.354353,0,1,0


In [63]:
data=data.drop(columns=['matchID', 'Court','cumDist','cumY','cumX','distPF','yPF','xPF'])


In [64]:
print(data.describe())

       min_fromNet  avg_fromNet  max_fromNet  std_fromNet  min_fromCentre  \
count   152.000000   152.000000   152.000000   152.000000      152.000000   
mean    817.619511  1186.485922  1419.012352   183.661186       17.335013   
std     376.600888   170.115309   116.058473   123.615433       55.732223   
min      69.626702   760.640733  1228.166417    32.602326        0.000000   
25%     437.334664  1077.081347  1327.990627    77.421017        0.492416   
50%    1027.667367  1222.028786  1415.155340   142.813390        1.433576   
75%    1140.918602  1329.696812  1501.825393   281.501893        4.083583   
max    1429.932454  1502.330037  1698.907824   542.643022      376.032948   

       avg_fromCentre  max_fromCentret  std_fromCentret    avg_dist  \
count      152.000000       152.000000       152.000000  152.000000   
mean       189.282812       397.262219       107.763995   12.525108   
std         81.496834       134.649714        39.807833    2.982561   
min         49.851511 

In [65]:
...
# Split-out validation dataset
X = data.drop("Winner", axis=1) # drop labels for the training set
y = data["Winner"].copy()
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [66]:
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder



In [67]:

numeric_features = ['min_fromNet',
 'avg_fromNet',
 'max_fromNet',
 'std_fromNet',
 'min_fromCentre',
 'avg_fromCentre',
 'max_fromCentret',
 'std_fromCentret',
 'avg_dist',
 'max_dist',
 'std_dist',
 'avg_Acceleration',
 'max_Acceleration',
 'std_Acceleration',
 'frontHalf',
 'backHalf',
 'behindLine']
numeric_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# categorical_features = ['Gender','Server']
# categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)])

In [68]:
...

# Spot Check Algorithms
models = []
models.append(('dum', DummyClassifier()))
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('SVM', SVC(gamma='scale')))
models.append(('SGD', SGDClassifier(loss='modified_huber')))
# evaluate each model in turn
results = []
names = []

for name, model in models:
    kfold = StratifiedKFold(n_splits=10, random_state=0, shuffle=True)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

/opt/anaconda3/lib/python3.8/site-packages/sklearn/dummy.py:131: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  warnings.warn("The default value of strategy will change from "


dum: 0.480769 (0.131215)
LR: 0.554487 (0.094787)
LDA: 0.554487 (0.120583)
KNN: 0.480128 (0.145174)
CART: 0.454487 (0.075600)
RF: 0.553846 (0.117963)
SVM: 0.544872 (0.126430)
SGD: 0.503846 (0.011538)


In [69]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
result=[]
for name,model in models:
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', model)])
    clf.fit(X_train,Y_train)
    predictions=clf.predict(X_test)
    acc=accuracy_score(Y_test, predictions)
    con=confusion_matrix(Y_test, predictions)
    rep=classification_report(Y_test, predictions)
    print('%s: acc= %s, con= %s, rep= %s' % (name, acc, con,rep))

dum: acc= 0.45161290322580644, con= [[8 9]
 [8 6]], rep=               precision    recall  f1-score   support

           0       0.50      0.47      0.48        17
           1       0.40      0.43      0.41        14

    accuracy                           0.45        31
   macro avg       0.45      0.45      0.45        31
weighted avg       0.45      0.45      0.45        31

LR: acc= 0.6774193548387096, con= [[10  7]
 [ 3 11]], rep=               precision    recall  f1-score   support

           0       0.77      0.59      0.67        17
           1       0.61      0.79      0.69        14

    accuracy                           0.68        31
   macro avg       0.69      0.69      0.68        31
weighted avg       0.70      0.68      0.68        31

LDA: acc= 0.6129032258064516, con= [[10  7]
 [ 5  9]], rep=               precision    recall  f1-score   support

           0       0.67      0.59      0.62        17
           1       0.56      0.64      0.60        14

    ac

/opt/anaconda3/lib/python3.8/site-packages/sklearn/dummy.py:131: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  warnings.warn("The default value of strategy will change from "


KNN: acc= 0.4838709677419355, con= [[8 9]
 [7 7]], rep=               precision    recall  f1-score   support

           0       0.53      0.47      0.50        17
           1       0.44      0.50      0.47        14

    accuracy                           0.48        31
   macro avg       0.49      0.49      0.48        31
weighted avg       0.49      0.48      0.48        31

CART: acc= 0.5483870967741935, con= [[9 8]
 [6 8]], rep=               precision    recall  f1-score   support

           0       0.60      0.53      0.56        17
           1       0.50      0.57      0.53        14

    accuracy                           0.55        31
   macro avg       0.55      0.55      0.55        31
weighted avg       0.55      0.55      0.55        31

RF: acc= 0.5483870967741935, con= [[10  7]
 [ 7  7]], rep=               precision    recall  f1-score   support

           0       0.59      0.59      0.59        17
           1       0.50      0.50      0.50        14

    accura

In [20]:
# Evaluate predictions
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

print(accuracy_score(Y_test, predictions))
print(confusion_matrix(Y_test, predictions))
print(classification_report(Y_test, predictions))

0.4782608695652174
[[ 0 12]
 [ 0 11]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        12
           1       0.48      1.00      0.65        11

    accuracy                           0.48        23
   macro avg       0.24      0.50      0.32        23
weighted avg       0.23      0.48      0.31        23

